In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

In [ ]:
from network_cons import *
import matplotlib.pyplot as plt

from utilities import growth_network_param_extractor

from stirling_approx import pairing_cond_num_asym, total_pairing_num, pairing_cond_num_asym_norm_fact
from stirling_approx import pairing_cond_num_asym_asym_h

from scipy.optimize import minimize_scalar

import pandas as pd

In [ ]:
from utilities import growth_network_param_extractor

In [ ]:
from stirling_approx import pairing_cond_num_asym, total_pairing_num, pairing_cond_num_asym_norm_fact
from stirling_approx import pairing_cond_num_asym_asym_h

In [ ]:
from scipy.optimize import minimize_scalar

In [ ]:
def normalized_likelihood( h ):
    return pairing_cond_num_asym( s_m, s_f, phi.sum(), h ) / pairing_cond_num_asym_norm_fact( s_m, s_f, h )

def log_normalized_likelihood(h):
    return -float( normalized_likelihood(h).log10() )

# Proportion imbalance

In [ ]:
mode_num = 2
proportion = 0.5
k_init = 4
late_comer_mode = 0
t_emerge = 0
t_unbias = 0
distribution = 'identical'
history_step_size = 0
eps = 1
inter_group_pair_num_output = True
inter_group_per_node = True

step_num = 2000

actual_h = 0.5

P_A = False

degree_symmetry = True
k_init = 4
k_init_2 = 4

In [ ]:
generative_h_range = np.linspace(0.1, 0.9, 17)
proportion_range = np.linspace(0.1, 0.5, 5)

In [ ]:
generative_h_range

In [ ]:
run_num = 2

In [ ]:
chm_calc = True

In [ ]:


# Initialize dataframe to store results

numberOfRows = run_num * len( generative_h_range ) * len(proportion_range)

EI_results = pd.DataFrame(columns=['actual_h', 'run'\
                    , 'proportion', 'EI']\
                    , index=np.arange(0, numberOfRows))
if chm_calc:
    EI_results['chm_infer_h'] = 0

In [ ]:
row_index = 0

for proportion in proportion_range:
    print(proportion)

    for h_i, actual_h in enumerate(generative_h_range):

        for run in range( run_num ):
            
            degrees, modes, external_links, phi = net_grow(step_num, mode_num, proportion, actual_h, k_init, late_comer_mode,
                           t_emerge, t_unbias, distribution, history_step_size, eps,
                           inter_group_pair_num_output, inter_group_per_node = True, symmetric=True, h2=None, degree_symmetry=degree_symmetry, k_init_2=k_init_2, P_A=P_A)

            E = external_links
            I = degrees - external_links

            EI = ( ( E - I ) / (E + I) ).mean()
            
            
            if chm_calc:
                s_f, s_m, phi = growth_network_param_extractor(degrees, modes, phi)
                result = minimize_scalar(log_normalized_likelihood, method='Bounded', bounds=[0.01, 0.99])
                chm_infer_h = result.x

            if chm_calc:
                EI_results.loc[row_index] = [actual_h, run, proportion, EI, chm_infer_h]
            else:
                EI_results.loc[row_index] = [actual_h, run, proportion, EI]
            row_index += 1


In [ ]:
result.x

In [ ]:
EI_results

In [ ]:
EI_results.astype('float').groupby(['actual_h', 'proportion']).mean().reset_index().drop( columns = 'run' )

In [ ]:
EI_results.groupby(['actual_h', 'proportion']).mean()

In [ ]:
EI_results.groupby(['actual_h', 'proportion']).mean().reset_index()

In [ ]:
EI_results_summary = EI_results.groupby(['actual_h', 'proportion']).mean().reset_index().drop(columns = 'run')

EI_results_summary['sem'] = EI_results.groupby(['actual_h', 'proportion']).sem().reset_index()['EI']

In [ ]:
EI_results_summary.to_csv('temp-EI_results_summary' + '-PA=' + str(P_A) + '.csv', index = False)

In [ ]:
EI_results_summary.to_csv('EI-homophily-results/EI_results_summary' + '-PA=' + str(P_A) + '.csv', index = False)

# Degree imbalance

In [ ]:
run_num = 100
proportion = 0.2
degree_symmetry = False
P_A = False

In [ ]:



# Initialize dataframe to store results
# degree_range = [1, 4, 8, 12, 40]
# degree_range = [1, 4, 40]
degree_range = [1, 4, 12]

numberOfRows = run_num * len( generative_h_range ) * len(degree_range)

EI_results_degree_imb = pd.DataFrame(columns=['actual_h', 'run'\
                    , 'k_init_2', 'EI']\
                    , index=np.arange(0, numberOfRows))


In [ ]:
row_index = 0

for k_init_2 in degree_range:
    print(k_init_2)

    for h_i, actual_h in enumerate(generative_h_range):

        for run in range( run_num ):
            
            degrees, modes, external_links, phi = net_grow(step_num, mode_num, proportion, actual_h, k_init, late_comer_mode,
                           t_emerge, t_unbias, distribution, history_step_size, eps,
                           inter_group_pair_num_output, inter_group_per_node = True, symmetric=True, h2=None, degree_symmetry=degree_symmetry, k_init_2=k_init_2, P_A=P_A)

            E = external_links
            I = degrees - external_links

            EI = ( ( E - I ) / (E + I) ).mean()

            EI_results_degree_imb.loc[row_index] = [actual_h, run, k_init_2, EI]
            row_index += 1


In [ ]:
EI_results_degree_imb_summary = EI_results_degree_imb.groupby(['actual_h', 'k_init_2']).mean().reset_index().drop(columns = 'run')

EI_results_degree_imb_summary['sem'] = EI_results_degree_imb.groupby(['actual_h', 'k_init_2']).sem().reset_index()['EI']

In [ ]:
EI_results_degree_imb_summary.to_csv('EI_results_summary_deg_imb' + '-PA=' + str(P_A) + '.csv', index = False)

In [ ]:
# EI_results_summary = EI_results.groupby(['actual_h', 'proportion']).mean().reset_index().drop(columns = 'run')

# EI_results_summary['sem'] = EI_results.groupby(['actual_h', 'proportion']).sem().reset_index()['EI']